In [1]:
import pandas as pd
import os
import itertools

In [2]:
IMAGE_SIZE = (1242, 375)
CAR_VAN_TRUCK = pd.read_csv("./CAR_VAN_TRUCK.csv")
CAR_VAN_TRUCK

,index,className
0,0,Car
1,1,Car
2,10,Car
3,100,Car
4,101,Car
...,...,...
127,94,Car
128,95,Car
129,96,Car
130,97,Car


In [3]:
def make_index(path):
    tmp = path.split('/')
    sceneNo = tmp[-3][-2:]
    sceneOpt = {'15-deg-left' : '00',
                '15-deg-right' : '01',
                '30-deg-left' : '02',
                '30-deg-right' : '03',
                'clone' : '04',
                'fog' : '05',
                'morning' : '06',
                'overcast' : '07',
                'rain' : '08',
                'sunset' : '09'
    }
    return sceneNo + sceneOpt[tmp[-2]]

In [4]:
def convert(bbox_path,save_path):
    #파일 불러오기
    df = pd.read_csv(bbox_path, sep=" ", header=None)
    df = df.drop([7, 8, 9, 10], axis=1)
    df = df.drop([0], axis=0)

    #kitti2yolo format 변환
    df['x'] = ((df[3].astype(int) + df[4].astype(int))/2.0) * (1./IMAGE_SIZE[0])
    df['y'] = ((df[5].astype(int) + df[6].astype(int))/2.0) * (1./IMAGE_SIZE[1])
    df['w'] = (df[4].astype(int) - df[3].astype(int)) * (1./IMAGE_SIZE[0])
    df['h'] = (df[6].astype(int) - df[5].astype(int)) * (1./IMAGE_SIZE[1])
    df = df.drop([3, 4, 5, 6], axis=1)

    #classNo 변환 (Car : 0, Van : 1, Truck : 2)
    car_list = CAR_VAN_TRUCK[CAR_VAN_TRUCK['className'] == 'Car'].drop(['className'], axis=1).values.tolist()
    van_list = CAR_VAN_TRUCK[CAR_VAN_TRUCK['className'] == 'Van'].drop(['className'], axis=1).values.tolist()
    truck_list = CAR_VAN_TRUCK[CAR_VAN_TRUCK['className'] == 'Truck'].drop(['className'], axis=1).values.tolist()

    car_list = list(itertools.chain(*car_list))
    van_list = list(itertools.chain(*van_list))
    truck_list = list(itertools.chain(*truck_list))

    for i in car_list:
        df.loc[df[2] == str(i),'classNo'] = '0'
    for i in van_list:
        df.loc[df[2] == str(i),'classNo'] = '1'
    for i in truck_list:
        df.loc[df[2] == str(i),'classNo'] = '2'
    
    df[2] = df['classNo']
    df = df.drop(['classNo'], axis=1)

    #프레임별 분리
    frame = []
    for i in range(int(df.loc[len(df)-1][0])+1):
        frame.append(df[df[0] == str(i)])

    #스테레오 카메라별 분리
    camera0, camera1 = [], []
    for df in frame:
        camera0.append(df[df[1] == '0'].drop([0,1], axis=1))
        camera1.append(df[df[1] == '1'].drop([0,1], axis=1))
    
    # df 저장
    for i, frame in enumerate(camera0):
        index = str(i).zfill(6)
        frame.to_csv('{}{}00{}.txt'.format(save_path,make_index(bbox_path),index), 
                    sep=' ', header=False, index=False)
    for i, frame in enumerate(camera1):
        index = str(i).zfill(6)
        frame.to_csv('{}{}01{}.txt'.format(save_path,make_index(bbox_path),index), 
                    sep=' ', header=False, index=False)


# save_bbox_path = '/usr/src/datasets/vkitti2.0.3/labels/'
# data_path = '/usr/src/datasets/rawVkitti/vkitti2.0.3bbox/Scene01/15-deg-left/bbox.txt'
# convert(data_path,save_bbox_path)

In [10]:
data_path = '/usr/src/datasets/rawVkitti/vkitti2.0.3rgb'
save_labels_path = '/usr/src/datasets/vkitti2.0.3/labels/'# 심볼릭링크가 다 모여질 폴더

vkitti_bbox_path = '/usr/src/datasets/rawVkitti/vkitti2.0.3bbox'

bbox_list = []
for (path, dir, files) in os.walk(vkitti_bbox_path):
    for filename in files:
        ext = os.path.basename(filename)   
        pathOpt = path.split('/')
        if ext == 'bbox.txt' and pathOpt[-1] == 'clone':
            bbox_list.append("%s/%s" % (path, filename))
print(bbox_list)

for bbox_path in bbox_list:
    convert(bbox_path, save_labels_path)
print('bbox converted to yolov5 format')
print('saved at %s' % (save_labels_path))

['/usr/src/datasets/rawVkitti/vkitti2.0.3bbox/Scene20/clone/bbox.txt', '/usr/src/datasets/rawVkitti/vkitti2.0.3bbox/Scene02/clone/bbox.txt', '/usr/src/datasets/rawVkitti/vkitti2.0.3bbox/Scene06/clone/bbox.txt', '/usr/src/datasets/rawVkitti/vkitti2.0.3bbox/Scene18/clone/bbox.txt', '/usr/src/datasets/rawVkitti/vkitti2.0.3bbox/Scene01/clone/bbox.txt']
bbox converted to yolov5 format


ValueError: incomplete format

In [11]:
index = make_index(data_path)

index

'0100'

In [2]:
# save_image_path = '/usr/src/datasets/vkitti2.0.3/images'

# # 이미지 cp
# cam0_path = '/usr/src/datasets/rawVkitti/vkitti2.0.3rgb/Scene01/15-deg-left/frames/rgb/Camera_0'
# cam1_path = '/usr/src/datasets/rawVkitti/vkitti2.0.3rgb/Scene01/15-deg-left/frames/rgb/Camera_1'
# label_path = '/usr/src/datasets/vkitti2.0.3/labels'

# image_list_cam0 = os.listdir(cam0_path)
# image_list_cam0.sort()
# image_list_cam1 = os.listdir(cam1_path)
# image_list_cam1.sort()

# for i, image in enumerate(image_list_cam0):
#     os.symlink(cam0_path+'/'+image, save_image_path+'/'+'0100'+'00'+str(i).zfill(6)+'.jpg')
# for i, image in enumerate(image_list_cam1):
#     os.symlink(cam1_path+'/'+image, save_image_path+'/'+'0100'+'01'+str(i).zfill(6)+'.jpg')